In [1]:
import torch
from torchvision import datasets, transforms
from fgsm import FGSM
import numpy as np

In [ ]:
# Set device to use (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define mean and standard deviation of CIFAR-10 dataset
mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]

# Define normalize transformation for input images
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, std)])


In [ ]:
# Load CIFAR-10 dataset
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)

In [ ]:
# Load pre-trained model
model = torch.load('model.pt').eval().to(device)

In [ ]:
# Instantiate FGSM attack
data_min = np.min((0 - np.array(mean)) /
                      np.array(std))
data_max = np.max((1 - np.array(mean)) /
                    np.array(std))
attack = FGSM(data_min, data_max, epsilon=0.3 * 255, num_steps=50, alpha=0.01, early_stop=True)

In [ ]:
total = 0
successes = 0

# Test the model on the dataset with and without attacks
for data, target in testloader:
    # Send data and target tensors to device
    data, target = data.to(device), target.to(device)

    # Generate adversarial example using FGSM attack
    adv_inputs, success, _ = attack(model, data, target)

    # Calculate prediction of original and adversarial input
    output = model(data)
    adv_output = model(adv_inputs)

    # Check if attack was successful and add to successes
    if success:
        successes += 1

    # Increment total number of predictions
    total += 1

# Calculate and print attack success rate
attack_success_rate = (successes / total) * 100
print('FGSM attack success rate on CIFAR-10 dataset: {:.2f}%'.format(attack_success_rate))